## First: Install Midiapipe

In [1]:
# !pip install mediapipe
# !pip install pynput

In [1]:
import cv2
import time
import os
import mediapipe as mp
import pyvjoy
from directkeys import PressKey, A, D, W, S, ReleaseKey

# Class for detection of hands

In [2]:
def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [15]:
def chooseImage(img, image_path):
    s_img = cv2.imread(image_path, -1)
    s_img = cv2.resize(s_img,(300, 300))
                         
    x_offset=y_offset=20
    y1, y2 = y_offset, y_offset + s_img.shape[0]
    x1, x2 = x_offset, x_offset + s_img.shape[1]
    alpha_s = s_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    for c in range(0, 3):
        img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] +
                                  alpha_l * img[y1:y2, x1:x2, c])
    return img

In [4]:
def getCommand(hands):
    command = "stop.png"
    if len(hands) == 0:
        ReleaseKey(D)
        ReleaseKey(A)
        ReleaseKey(S)
        ReleaseKey(W)
        
    if len(hands) == 2:
        handOneOpen = hands[0][0]>2
        handOneDirection = hands[0][1] == "Left"
        
        handTwoOpen = hands[1][0]>2
        handTwoDirection = hands[1][1] == "Right"
        
        if(handOneOpen and handTwoOpen):
            command = "speed.png"
            ReleaseKey(D)
            ReleaseKey(A)
            ReleaseKey(S)
            PressKey(W)
        else:
            if(handOneOpen and handOneDirection):
                command = "left.png"
                ReleaseKey(W)
                ReleaseKey(D)
                ReleaseKey(S)
                PressKey(A)
            if(handTwoOpen and handTwoDirection):
                command = "right.png"
                ReleaseKey(W)
                ReleaseKey(A)
                ReleaseKey(S)
                PressKey(D)
            if(not(handOneOpen) and not(handTwoOpen)):
                command = "come_back.png"
                ReleaseKey(W)
                ReleaseKey(D)
                ReleaseKey(A)
                PressKey(S)
    
    return "./imagens_cars/"+command

In [5]:
class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
 
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
 
    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        
        num_hands = 0
        if self.results.multi_handedness:
            for handLms in self.results.multi_handedness:
                num_hands = num_hands+1
            num_hands = num_hands-1
    
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)
        return img, num_hands
 
    def findPosition(self, img, num_hands=0, draw=True):
        hands = []
        for i in range(num_hands+1):
            lmList = []
            if self.results.multi_hand_landmarks:
                left_right = ""
                left_right = self.results.multi_handedness[i].classification[0].label
                
                myHand = self.results.multi_hand_landmarks[i]
                
                for id, lm in enumerate(myHand.landmark):
                    # print(id, lm)
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    # print(id, cx, cy)
                    lmList.append([id, cx, cy])
                    if draw:
                        cv2.circle(img, (cx, cy), 105, (255, 0, 255), cv2.FILLED)
                    
                hands.append([lmList, left_right])
        return hands

In [6]:
def checkHand(num_hands):
    hands = detector.findPosition(img, num_hands=num_hands, draw=False)
    detection = []
    for i in hands:
        lmList = i[0]
        if len(lmList) != 0:
            fingers = []
            # Thumb
            if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
            # 4 Fingers
            for id in range(1, 5):
                if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            totalFingers = fingers.count(1)
            detection.append([totalFingers, i[1]])
            
    return detection

## Configurations of video

In [7]:
# Configurations
fontColor = (255, 0, 0);
fontSize = 2;
fontBody = 3;
FPSPosition = (5, 30);
NumberPosition = (5, 90);

pTime = 0
tipIds = [4, 8, 12, 16, 20]

detector = handDetector(detectionCon=0.75)

## Start the application

In [22]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # captures the video frame
    success, img = cap.read()
    img, num_hands = detector.findHands(img)
    hands = checkHand(num_hands)
    img = chooseImage(img, getCommand(hands));
    
    img = rescale_frame(img, percent=50)
    cv2.imshow("Image", img)
    
    # Press "Q" to close windows-video
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()